# ALIBI
## Load data

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd

df = pd.read_csv("../../datasets/FICO/heloc_dataset_v1.csv")

## XGBoost

### Load model

In [3]:
from xgboost.sklearn import XGBClassifier

model = XGBClassifier()

In [4]:
model.load_model("../../models/xgboost.json")

### Standard counterfactuals

In [5]:
from alibi.explainers import Counterfactual

In [6]:
shape = (1, 23)

In [7]:
predict_fn = lambda x: model.predict_proba(x)

In [8]:
import tensorflow as tf

tf.keras.backend.clear_session()
tf.compat.v1.disable_eager_execution()
tf.compat.v1.disable_v2_behavior()

Instructions for updating:
non-resource variables are not supported in the long term


In [9]:
cf = Counterfactual(predict_fn, shape, distance_fn='l1', target_proba=0.75,
                    target_class="other", max_iter=1000, early_stop=100, lam_init=1e-1,
                    max_lam_steps=100, tol=0.25, learning_rate_init=0.7,
                    feature_range=(-20, 1000), eps=0.5, init='identity',
                    decay=True, write_dir=None, debug=False)

2022-03-16 15:56:28.641570: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [10]:
import utils

X = utils.get_negative_closest(model, 0.75).to_numpy().reshape(1, -1)

In [11]:
explanation = cf.explain(X)

## Export results

In [12]:
N_cf = len(explanation['data']['all'][1])
schema = utils.get_result_schema()
v = [[] for i in range(23)]
o = [[] for i in range(23)]
goalValue = []
goalScore = []
for i in range(N_cf):
    ccf = explanation['data']['all'][1][i]
    values = ccf['X'].tolist()[0]
    goalValue.append(ccf['class'])
    goalScore.append(ccf['proba'].tolist()[0][1])
    for j in range(23):
        v[j].append(values[j])
        o[j].append(X[0][j])
    
d = {e[0]: e[1] for e in zip(schema[23:-4], v)}
od = {e[0]: e[1] for e in zip(schema[:23], o)}
goal = {"GoalValue": goalValue, "GoalScore": goalScore, "GoalName": ["RiskPerformance"] * N_cf}
method = ["Alibi-Standard"] * N_cf

result = utils.save_result(original=od, cf=d, score=goal, method=method, model="XGBoost")
result.iloc[:,23:].describe().T

,count,mean,std,min,25%,50%,75%,max
CfExternalRiskEstimate,100.0,77.093906,1.892271,70.880516,76.915333,77.980034,78.127724,78.687592
CfMSinceOldestTradeOpen,100.0,43.000000,0.000000,43.000000,43.000000,43.000000,43.000000,43.000000
CfMSinceMostRecentTradeOpen,100.0,2.000000,0.000000,2.000000,2.000000,2.000000,2.000000,2.000000
CfAverageMInFile,100.0,26.000000,0.000000,26.000000,26.000000,26.000000,26.000000,26.000000
CfNumSatisfactoryTrades,100.0,15.221119,0.331481,14.757203,14.977712,15.147840,15.314057,16.023129
CfNumTrades60Ever2DerogPubRec,100.0,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000
CfNumTrades90Ever2DerogPubRec,100.0,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000
CfPercentTradesNeverDelq,100.0,100.000000,0.000000,100.000000,100.000000,100.000000,100.000000,100.000000
CfMSinceMostRecentDelq,100.0,-7.000000,0.000000,-7.000000,-7.000000,-7.000000,-7.000000,-7.000000
CfMaxDelq2PublicRecLast12M,100.0,7.000000,0.000000,7.000000,7.000000,7.000000,7.000000,7.000000


In [13]:
result.to_csv("../../results/cf-alibi.csv")